### 180

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
# df_hist = utils.read_df_pkl('../input/hist_exist*')
df_hist = reduce_mem_usage(df_hist)
# df_new = utils.read_df_pkl('../input/new_mer*0*')
df_new = utils.read_df_pkl('../input/new_clean_rdm0*')
# df_new = utils.read_df_pkl('../input/new_exist*')
df_new = reduce_mem_usage(df_new)
df_trans = pd.concat([df_hist, df_new], axis=0)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_trans.shape)
df_trans = df_trans.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_trans.shape)

df = df_trans
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

  0%|          | 0/3 [00:00<?, ?it/s]

Mem. usage decreased to 2554.26 Mb (0.0% reduction)


100%|██████████| 3/3 [00:01<00:00,  2.70it/s]


Mem. usage decreased to 183.47 Mb (0.0% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(31075392, 21)
(31075392, 22)


### New Merchant

In [8]:
for past1, past2 in zip(np.arange(7), np.arange(7)+1):
    print(past1, past2, past2+1, past2+2)
sys.exit()

0 1 2 3
1 2 3 4
2 3 4 5
3 4 5 6
4 5 6 7
5 6 7 8
6 7 8 9


SystemExit: 

In [26]:
m_id = 'merchant_id'
mc_id = 'merchant_category_id'
lag_col = 'org_month_lag'
past_1 = 0
past_2 = 3
future_1 = 4
future_2 = 5

past_2_list = np.array([3,4,5,6,7,8,9,10])
past_1_list = np.zeros(len(past_2_list))

future_1_list = past_2_list.copy() + 1
future_2_list = future_1_list.copy() + 1

feat_list = []
fname_list = [ f'auth1_new_' ,f'auth0_new_']
    

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

    
def feat_agg(df, fname):
# for df, fname in zip(df_list, fname_list):
    global train_test
    
    aggs = {}
    if fname.count('cat1'):
        df = pd.get_dummies(df, columns=['category_3'])
        aggs = {
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    else:
        df = pd.get_dummies(df, columns=['category_2', 'category_3'])
        aggs = {
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    
    if not(fname.count('cat')):
        aggs['category_1'] = ['sum', 'mean']
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_this_term'] =  (pd.to_datetime(df_agg[f'{fname}_purchase_date_max']) - pd.to_datetime(df_agg[f'{fname}_purchase_date_min'])).dt.days
    df_agg[f'{fname}_freq_per_this_term'] = df_agg[f'{fname}_card_id_size']              / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_amount_per_this_term'] = df_agg[f'{fname}_purchase_amount_sum']     / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_instthisments_per_this_term'] = df_agg[f'{fname}_installments_sum'] / (df_agg[f'{fname}_this_term'] + 1.0)
    
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    monthly=True
#     monthly=False
    
#     Monthly Agg
    if monthly:
        new_columns = get_new_columns(fname + '_monthly_avg', aggs)
        month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
            
        month_agg.columns = new_columns
        month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
        month_agg.reset_index(drop=False,inplace=True)
        
        train_test = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
        
        del df_agg, month_agg
        gc.collect()
    else:
        train_test = train_test.join(df_agg.set_index(key))
        del df_agg
        gc.collect()
    
    train_test = impute_agg(train_test)
    train_test.columns =  [col.replace('.', '_') for col in train_test.columns]
    
    display(train_test.head(3))
    
    feat_list.append(train_test.copy())
    del train_test
    gc.collect()
    
#     print(f"Saving features... {fname} {train_test.shape}")
#     p_list = Parallel(n_jobs=-1)([delayed(save_feature)(col) for col in train_test.columns])

#     for col in tqdm(train_test.columns):
#         ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
#         if col in ignore_features: return
#         if (col.count('feature_')):return
#         if (col.count('purchase_date')):return
#         feature = train_test[col].fillna(-1).astype('float32').values
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)


def save_feature(col):
    ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
    if col in ignore_features: return
    if (col.count('feature_')):return
    if (col.count('purchase_date')):return
    feature = train_test[col].fillna(-1).astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
    

def impute_agg(df):
    for col in df.columns:
        if col.count('category_1'):
            df[col].fillna(-1, inplace=True)
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df
    
    
# for df, fname in zip([auth1, auth0], fname_list):
for df, fname_base in zip([auth1], fname_list):
    
    for past_1, past_2 in zip(past_1_list, past_2_list):
        
        future_1 = past_2+1
        future_2 = past_2+2
#         fname = f'180_l{future_1}{future_2}'
        fname = fname_base + f'org_lag{future_1}_{future_2}'

        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        df_train.set_index(key, inplace=True)
        df_test.set_index(key, inplace=True)
        train_test = pd.concat([df_train, df_test], axis=0)
        train_test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        
        future = df[future_1<= df[lag_col]][df[lag_col]<=future_2][[key, mc_id, m_id]].drop_duplicates()
        past = df[past_1<= df[lag_col]][df[lag_col]<=past_2][[key, mc_id, m_id]].drop_duplicates()
        
        # inner joinで過去に存在したmerchantをdf_nowに残し、そのindexをdf_nowから除外することでnew_merchantのみ残す。天才！
        future['index'] = np.arange(len(future))
        mer_same = future.merge(past, how='inner', on=[key, m_id, mc_id])
        past_mer_index = mer_same['index'].values
        future = future.loc[~future['index'].isin(past_mer_index), :]
        
        future.drop('index', axis=1, inplace=True)
        future.dropna(inplace=True)
        
        # ここでtransactionをinner joinしてnew_merchantに対する購買履歴のみのテーブルができる
        new_hist = future.merge(df, how='inner', on=[key, mc_id, m_id])
        
        feat_agg(new_hist, fname)

        
# for df in feat_list:
#     try:
#         df.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
#     except KeyError:
#         pass
    
#     try:
#         df.set_index(key, inplace=True)
#     except KeyError:
#         pass
    
#     train_test = train_test.join(df)

# train_test = impute_agg(train_test)
# display(train_test.head())

100%|██████████| 3/3 [00:00<00:00, 223.44it/s]


,auth1_new_org_lag4_5_category_2_1_0_mean,auth1_new_org_lag4_5_category_2_2_0_mean,auth1_new_org_lag4_5_category_2_3_0_mean,auth1_new_org_lag4_5_category_2_4_0_mean,auth1_new_org_lag4_5_category_2_5_0_mean,auth1_new_org_lag4_5_category_3_A_mean,auth1_new_org_lag4_5_category_3_B_mean,auth1_new_org_lag4_5_category_3_C_mean,auth1_new_org_lag4_5_category_1_sum,auth1_new_org_lag4_5_category_1_mean,auth1_new_org_lag4_5_month_lag_mean,auth1_new_org_lag4_5_month_lag_std,auth1_new_org_lag4_5_yyyy_week_nunique,auth1_new_org_lag4_5_purchase_date_max,auth1_new_org_lag4_5_purchase_date_min,auth1_new_org_lag4_5_month_diff_mean,auth1_new_org_lag4_5_month_diff_std,auth1_new_org_lag4_5_purchase_amount_sum,auth1_new_org_lag4_5_purchase_amount_max,auth1_new_org_lag4_5_purchase_amount_min,auth1_new_org_lag4_5_purchase_amount_mean,auth1_new_org_lag4_5_installments_sum,auth1_new_org_lag4_5_installments_max,auth1_new_org_lag4_5_installments_min,auth1_new_org_lag4_5_installments_mean,auth1_new_org_lag4_5_installments_std,auth1_new_org_lag4_5_merchant_id_nunique,auth1_new_org_lag4_5_merchant_category_id_nunique,auth1_new_org_lag4_5_card_id_size,auth1_new_org_lag4_5_city_id_nunique,auth1_new_org_lag4_5_state_id_nunique,auth1_new_org_lag4_5_subsector_id_nunique,auth1_new_org_lag4_5_this_term,auth1_new_org_lag4_5_freq_per_this_term,auth1_new_org_lag4_5_amount_per_this_term,auth1_new_org_lag4_5_instthisments_per_this_term,auth1_new_org_lag4_5_amount_per_installments_sum,auth1_new_org_lag4_5_amount_per_installments_mean,auth1_new_org_lag4_5_monthly_avg_category_2_1_0_mean,auth1_new_org_lag4_5_monthly_avg_category_2_2_0_mean,auth1_new_org_lag4_5_monthly_avg_category_2_3_0_mean,auth1_new_org_lag4_5_monthly_avg_category_2_4_0_mean,auth1_new_org_lag4_5_monthly_avg_category_2_5_0_mean,auth1_new_org_lag4_5_monthly_avg_category_3_A_mean,auth1_new_org_lag4_5_monthly_avg_category_3_B_mean,auth1_new_org_lag4_5_monthly_avg_category_3_C_mean,auth1_new_org_lag4_5_monthly_avg_category_1_sum,auth1_new_org_lag4_5_monthly_avg_category_1_mean,auth1_new_org_lag4_5_monthly_avg_month_lag_mean,auth1_new_org_lag4_5_monthly_avg_month_lag_std,auth1_new_org_lag4_5_monthly_avg_yyyy_week_nunique,auth1_new_org_lag4_5_monthly_avg_month_diff_mean,auth1_new_org_lag4_5_monthly_avg_month_diff_std,auth1_new_org_lag4_5_monthly_avg_purchase_amount_sum,auth1_new_org_lag4_5_monthly_avg_purchase_amount_max,auth1_new_org_lag4_5_monthly_avg_purchase_amount_min,auth1_new_org_lag4_5_monthly_avg_purchase_amount_mean,auth1_new_org_lag4_5_monthly_avg_installments_sum,auth1_new_org_lag4_5_monthly_avg_installments_max,auth1_new_org_lag4_5_monthly_avg_installments_min,auth1_new_org_lag4_5_monthly_avg_installments_mean,auth1_new_org_lag4_5_monthly_avg_installments_std,auth1_new_org_lag4_5_monthly_avg_merchant_id_nunique,auth1_new_org_lag4_5_monthly_avg_merchant_category_id_nunique,auth1_new_org_lag4_5_monthly_avg_card_id_size,auth1_new_org_lag4_5_monthly_avg_city_id_nunique,auth1_new_org_lag4_5_monthly_avg_state_id_nunique,auth1_new_org_lag4_5_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000000,0.0,0.0,0.0,0.00000,1.0,0.000000,0.000000,0.0,0.000000,-2.848485,1.394660,14.0,2018-02-21 18:49:39,2017-10-01 15:38:43,161.545455,42.961531,3.566406,0.300537,0.010529,0.108032,0.0,0.0,0.0,0.000000,0.000000,18.0,15.0,33.0,3.0,1.0,13.0,143.0,0.229167,0.024767,0.000000,3.566406,0.108032,1.000000,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,-2.250000,0.0,3.750000,144.076786,8.921070,0.891602,0.279785,0.021866,0.117493,0.000000,0.000000,0.0,0.000000,0.000000,6.500000,5.750000,8.250000,1.500000,1.000000,5.500000
C_ID_3d0044924f,0.989474,0.0,0.0,0.0,0.00000,0.0,0.957895,0.042105,1.0,0.010526,-3.663158,3.106652,27.0,2018-01-31 22:31:09,2017-05-01 12:32:13,216.200000,95.689947,5.589844,1.083984,0.006771,0.058838,105.0,6.0,1.0,1.105469,0.592285,23.0,17.0,95.0,2.0,2.0,11.0,275.0,0.344203,0.020253,0.380435,0.052734,0.027939,0.99382

100%|██████████| 3/3 [00:00<00:00, 208.04it/s]


,auth1_new_org_lag5_6_category_2_1_0_mean,auth1_new_org_lag5_6_category_2_2_0_mean,auth1_new_org_lag5_6_category_2_3_0_mean,auth1_new_org_lag5_6_category_2_4_0_mean,auth1_new_org_lag5_6_category_2_5_0_mean,auth1_new_org_lag5_6_category_3_A_mean,auth1_new_org_lag5_6_category_3_B_mean,auth1_new_org_lag5_6_category_3_C_mean,auth1_new_org_lag5_6_category_1_sum,auth1_new_org_lag5_6_category_1_mean,auth1_new_org_lag5_6_month_lag_mean,auth1_new_org_lag5_6_month_lag_std,auth1_new_org_lag5_6_yyyy_week_nunique,auth1_new_org_lag5_6_purchase_date_max,auth1_new_org_lag5_6_purchase_date_min,auth1_new_org_lag5_6_month_diff_mean,auth1_new_org_lag5_6_month_diff_std,auth1_new_org_lag5_6_purchase_amount_sum,auth1_new_org_lag5_6_purchase_amount_max,auth1_new_org_lag5_6_purchase_amount_min,auth1_new_org_lag5_6_purchase_amount_mean,auth1_new_org_lag5_6_installments_sum,auth1_new_org_lag5_6_installments_max,auth1_new_org_lag5_6_installments_min,auth1_new_org_lag5_6_installments_mean,auth1_new_org_lag5_6_installments_std,auth1_new_org_lag5_6_merchant_id_nunique,auth1_new_org_lag5_6_merchant_category_id_nunique,auth1_new_org_lag5_6_card_id_size,auth1_new_org_lag5_6_city_id_nunique,auth1_new_org_lag5_6_state_id_nunique,auth1_new_org_lag5_6_subsector_id_nunique,auth1_new_org_lag5_6_this_term,auth1_new_org_lag5_6_freq_per_this_term,auth1_new_org_lag5_6_amount_per_this_term,auth1_new_org_lag5_6_instthisments_per_this_term,auth1_new_org_lag5_6_amount_per_installments_sum,auth1_new_org_lag5_6_amount_per_installments_mean,auth1_new_org_lag5_6_monthly_avg_category_2_1_0_mean,auth1_new_org_lag5_6_monthly_avg_category_2_2_0_mean,auth1_new_org_lag5_6_monthly_avg_category_2_3_0_mean,auth1_new_org_lag5_6_monthly_avg_category_2_4_0_mean,auth1_new_org_lag5_6_monthly_avg_category_2_5_0_mean,auth1_new_org_lag5_6_monthly_avg_category_3_A_mean,auth1_new_org_lag5_6_monthly_avg_category_3_B_mean,auth1_new_org_lag5_6_monthly_avg_category_3_C_mean,auth1_new_org_lag5_6_monthly_avg_category_1_sum,auth1_new_org_lag5_6_monthly_avg_category_1_mean,auth1_new_org_lag5_6_monthly_avg_month_lag_mean,auth1_new_org_lag5_6_monthly_avg_month_lag_std,auth1_new_org_lag5_6_monthly_avg_yyyy_week_nunique,auth1_new_org_lag5_6_monthly_avg_month_diff_mean,auth1_new_org_lag5_6_monthly_avg_month_diff_std,auth1_new_org_lag5_6_monthly_avg_purchase_amount_sum,auth1_new_org_lag5_6_monthly_avg_purchase_amount_max,auth1_new_org_lag5_6_monthly_avg_purchase_amount_min,auth1_new_org_lag5_6_monthly_avg_purchase_amount_mean,auth1_new_org_lag5_6_monthly_avg_installments_sum,auth1_new_org_lag5_6_monthly_avg_installments_max,auth1_new_org_lag5_6_monthly_avg_installments_min,auth1_new_org_lag5_6_monthly_avg_installments_mean,auth1_new_org_lag5_6_monthly_avg_installments_std,auth1_new_org_lag5_6_monthly_avg_merchant_id_nunique,auth1_new_org_lag5_6_monthly_avg_merchant_category_id_nunique,auth1_new_org_lag5_6_monthly_avg_card_id_size,auth1_new_org_lag5_6_monthly_avg_city_id_nunique,auth1_new_org_lag5_6_monthly_avg_state_id_nunique,auth1_new_org_lag5_6_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000000,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.0,0.000000,-1.930233,0.768280,12.0,2018-02-17 09:39:17,2017-11-11 00:00:00,131.279070,22.983768,9.406250,2.000000,0.012032,0.218872,0.0,0.0,0.0,0.000000,0.000000,31.0,20.0,43.0,1.0,1.0,14.0,98.0,0.434343,0.095013,0.000000,9.406250,0.218872,1.000000,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000,0.00000,-1.500000,0.0,3.250,118.096429,7.609796,2.353516,0.670410,0.049072,0.160156,0.0,0.000,0.0,0.000000,0.000000,9.250000,7.250000,10.750000,1.000000,1.000000,5.500000
C_ID_3d0044924f,0.990099,0.0,0.0,0.0,0.000000,0.0,0.940594,0.059406,1.0,0.009901,-3.475248,2.841106,26.0,2018-01-31 22:31:09,2017-06-03 02:58:07,211.603960,88.599219,7.070312,1.083984,0.006172,0.070007,116.0,6.0,1.0,1.148438,0.698242,28.0,15.0,101.0,3.0,2.0,11.0,242.0,0.415638,0.029096,0.477366,0.060425,0.032593,0.993750,0.0,0.0,0.

100%|██████████| 3/3 [00:00<00:00, 186.34it/s]


...

,auth1_new_org_lag9_10_category_2_1_0_mean,auth1_new_org_lag9_10_category_2_2_0_mean,auth1_new_org_lag9_10_category_2_3_0_mean,auth1_new_org_lag9_10_category_2_4_0_mean,auth1_new_org_lag9_10_category_2_5_0_mean,auth1_new_org_lag9_10_category_3_A_mean,auth1_new_org_lag9_10_category_3_B_mean,auth1_new_org_lag9_10_category_3_C_mean,auth1_new_org_lag9_10_category_1_sum,auth1_new_org_lag9_10_category_1_mean,auth1_new_org_lag9_10_month_lag_mean,auth1_new_org_lag9_10_month_lag_std,auth1_new_org_lag9_10_yyyy_week_nunique,auth1_new_org_lag9_10_purchase_date_max,auth1_new_org_lag9_10_purchase_date_min,auth1_new_org_lag9_10_month_diff_mean,auth1_new_org_lag9_10_month_diff_std,auth1_new_org_lag9_10_purchase_amount_sum,auth1_new_org_lag9_10_purchase_amount_max,auth1_new_org_lag9_10_purchase_amount_min,auth1_new_org_lag9_10_purchase_amount_mean,auth1_new_org_lag9_10_installments_sum,auth1_new_org_lag9_10_installments_max,auth1_new_org_lag9_10_installments_min,auth1_new_org_lag9_10_installments_mean,auth1_new_org_lag9_10_installments_std,auth1_new_org_lag9_10_merchant_id_nunique,auth1_new_org_lag9_10_merchant_category_id_nunique,auth1_new_org_lag9_10_card_id_size,auth1_new_org_lag9_10_city_id_nunique,auth1_new_org_lag9_10_state_id_nunique,auth1_new_org_lag9_10_subsector_id_nunique,auth1_new_org_lag9_10_this_term,auth1_new_org_lag9_10_freq_per_this_term,auth1_new_org_lag9_10_amount_per_this_term,auth1_new_org_lag9_10_instthisments_per_this_term,auth1_new_org_lag9_10_amount_per_installments_sum,auth1_new_org_lag9_10_amount_per_installments_mean,auth1_new_org_lag9_10_monthly_avg_category_2_1_0_mean,auth1_new_org_lag9_10_monthly_avg_category_2_2_0_mean,auth1_new_org_lag9_10_monthly_avg_category_2_3_0_mean,auth1_new_org_lag9_10_monthly_avg_category_2_4_0_mean,auth1_new_org_lag9_10_monthly_avg_category_2_5_0_mean,auth1_new_org_lag9_10_monthly_avg_category_3_A_mean,auth1_new_org_lag9_10_monthly_avg_category_3_B_mean,auth1_new_org_lag9_10_monthly_avg_category_3_C_mean,auth1_new_org_lag9_10_monthly_avg_category_1_sum,auth1_new_org_lag9_10_monthly_avg_category_1_mean,auth1_new_org_lag9_10_monthly_avg_month_lag_mean,auth1_new_org_lag9_10_monthly_avg_month_lag_std,auth1_new_org_lag9_10_monthly_avg_yyyy_week_nunique,auth1_new_org_lag9_10_monthly_avg_month_diff_mean,auth1_new_org_lag9_10_monthly_avg_month_diff_std,auth1_new_org_lag9_10_monthly_avg_purchase_amount_sum,auth1_new_org_lag9_10_monthly_avg_purchase_amount_max,auth1_new_org_lag9_10_monthly_avg_purchase_amount_min,auth1_new_org_lag9_10_monthly_avg_purchase_amount_mean,auth1_new_org_lag9_10_monthly_avg_installments_sum,auth1_new_org_lag9_10_monthly_avg_installments_max,auth1_new_org_lag9_10_monthly_avg_installments_min,auth1_new_org_lag9_10_monthly_avg_installments_mean,auth1_new_org_lag9_10_monthly_avg_installments_std,auth1_new_org_lag9_10_monthly_avg_merchant_id_nunique,auth1_new_org_lag9_10_monthly_avg_merchant_category_id_nunique,auth1_new_org_lag9_10_monthly_avg_card_id_size,auth1_new_org_lag9_10_monthly_avg_city_id_nunique,auth1_new_org_lag9_10_monthly_avg_state_id_nunique,auth1_new_org_lag9_10_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,1.478261,0.510754,7.0,2018-04-29 11:23:05,2018-03-05 14:04:36,29.739130,16.591941,3.939453,0.450928,0.022476,0.171265,0.0,0.0,0.0,0.000000,0.000000,23.0,14.0,23.0,3.0,1.0,10.0,54.0,0.418182,0.071626,0.000000,3.939453,0.171265,1.000000,0.0,0.0,0.0,0.0,1.0,0.00000,0.00000,0.00,0.000000,1.5,0.0,3.50,29.151515,9.355305,1.969727,0.445068,0.024918,0.170410,0.00,0.00,0.00,0.0000,0.000000,11.50,8.50,11.5,2.00,1.0,6.00
C_ID_3d0044924f,0.954545,0.0,0.0,0.0,0.0,0.0,0.863636,0.136364,1.0,0.045455,-2.590909,0.796366,9.0,2018-01-23 18:49:01,2017-10-15 17:20:41,179.136364,25.392716,2.396484,0.330322,0.006020,0.108887,29.0,4.0,1.0,1.318359,0.838867,19.0,14.0,22.0,4.0,3.0,9.0,100.0,0.217822,0.023728,0.287129,0.079895,0.046967,0.984375,

100%|██████████| 3/3 [00:00<00:00, 179.18it/s]


,auth1_new_org_lag10_11_category_2_1_0_mean,auth1_new_org_lag10_11_category_2_2_0_mean,auth1_new_org_lag10_11_category_2_3_0_mean,auth1_new_org_lag10_11_category_2_4_0_mean,auth1_new_org_lag10_11_category_2_5_0_mean,auth1_new_org_lag10_11_category_3_A_mean,auth1_new_org_lag10_11_category_3_B_mean,auth1_new_org_lag10_11_category_3_C_mean,auth1_new_org_lag10_11_category_1_sum,auth1_new_org_lag10_11_category_1_mean,auth1_new_org_lag10_11_month_lag_mean,auth1_new_org_lag10_11_month_lag_std,auth1_new_org_lag10_11_yyyy_week_nunique,auth1_new_org_lag10_11_purchase_date_max,auth1_new_org_lag10_11_purchase_date_min,auth1_new_org_lag10_11_month_diff_mean,auth1_new_org_lag10_11_month_diff_std,auth1_new_org_lag10_11_purchase_amount_sum,auth1_new_org_lag10_11_purchase_amount_max,auth1_new_org_lag10_11_purchase_amount_min,auth1_new_org_lag10_11_purchase_amount_mean,auth1_new_org_lag10_11_installments_sum,auth1_new_org_lag10_11_installments_max,auth1_new_org_lag10_11_installments_min,auth1_new_org_lag10_11_installments_mean,auth1_new_org_lag10_11_installments_std,auth1_new_org_lag10_11_merchant_id_nunique,auth1_new_org_lag10_11_merchant_category_id_nunique,auth1_new_org_lag10_11_card_id_size,auth1_new_org_lag10_11_city_id_nunique,auth1_new_org_lag10_11_state_id_nunique,auth1_new_org_lag10_11_subsector_id_nunique,auth1_new_org_lag10_11_this_term,auth1_new_org_lag10_11_freq_per_this_term,auth1_new_org_lag10_11_amount_per_this_term,auth1_new_org_lag10_11_instthisments_per_this_term,auth1_new_org_lag10_11_amount_per_installments_sum,auth1_new_org_lag10_11_amount_per_installments_mean,auth1_new_org_lag10_11_monthly_avg_category_2_1_0_mean,auth1_new_org_lag10_11_monthly_avg_category_2_2_0_mean,auth1_new_org_lag10_11_monthly_avg_category_2_3_0_mean,auth1_new_org_lag10_11_monthly_avg_category_2_4_0_mean,auth1_new_org_lag10_11_monthly_avg_category_2_5_0_mean,auth1_new_org_lag10_11_monthly_avg_category_3_A_mean,auth1_new_org_lag10_11_monthly_avg_category_3_B_mean,auth1_new_org_lag10_11_monthly_avg_category_3_C_mean,auth1_new_org_lag10_11_monthly_avg_category_1_sum,auth1_new_org_lag10_11_monthly_avg_category_1_mean,auth1_new_org_lag10_11_monthly_avg_month_lag_mean,auth1_new_org_lag10_11_monthly_avg_month_lag_std,auth1_new_org_lag10_11_monthly_avg_yyyy_week_nunique,auth1_new_org_lag10_11_monthly_avg_month_diff_mean,auth1_new_org_lag10_11_monthly_avg_month_diff_std,auth1_new_org_lag10_11_monthly_avg_purchase_amount_sum,auth1_new_org_lag10_11_monthly_avg_purchase_amount_max,auth1_new_org_lag10_11_monthly_avg_purchase_amount_min,auth1_new_org_lag10_11_monthly_avg_purchase_amount_mean,auth1_new_org_lag10_11_monthly_avg_installments_sum,auth1_new_org_lag10_11_monthly_avg_installments_max,auth1_new_org_lag10_11_monthly_avg_installments_min,auth1_new_org_lag10_11_monthly_avg_installments_mean,auth1_new_org_lag10_11_monthly_avg_installments_std,auth1_new_org_lag10_11_monthly_avg_merchant_id_nunique,auth1_new_org_lag10_11_monthly_avg_merchant_category_id_nunique,auth1_new_org_lag10_11_monthly_avg_card_id_size,auth1_new_org_lag10_11_monthly_avg_city_id_nunique,auth1_new_org_lag10_11_monthly_avg_state_id_nunique,auth1_new_org_lag10_11_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,2.000000,0.000000,4.0,2018-04-29 11:23:05,2018-04-06 10:55:08,15.636364,8.617108,1.664062,0.439209,0.027359,0.151245,0.0,0.0,0.0,0.000000,0.000000,11.0,9.0,11.0,1.0,1.0,6.0,23.0,0.458333,0.069336,0.000000,1.664062,0.151245,1.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,2.0,0.0,4.0,15.636364,8.617108,1.664062,0.439209,0.027359,0.151245,0.000000,0.0,0.000000,0.000000,0.000000,11.0,9.000000,11.000000,1.0,1.000000,6.000000
C_ID_3d0044924f,0.913043,0.0,0.0,0.0,0.0,0.0,0.782609,0.173913,2.0,0.086957,-1.130435,0.457697,9.0,2018-01-23 18:49:01,2017-11-06 00:00:00,134.391304,16.991279,1.747070,0.300537,0.004517,0.075989,29.0,4.0,0.0,1.260742,0

100%|██████████| 3/3 [00:00<00:00, 178.75it/s]


,auth1_new_org_lag11_12_category_2_1_0_mean,auth1_new_org_lag11_12_category_2_2_0_mean,auth1_new_org_lag11_12_category_2_3_0_mean,auth1_new_org_lag11_12_category_2_4_0_mean,auth1_new_org_lag11_12_category_2_5_0_mean,auth1_new_org_lag11_12_category_3_A_mean,auth1_new_org_lag11_12_category_3_B_mean,auth1_new_org_lag11_12_category_3_C_mean,auth1_new_org_lag11_12_category_1_sum,auth1_new_org_lag11_12_category_1_mean,auth1_new_org_lag11_12_month_lag_mean,auth1_new_org_lag11_12_month_lag_std,auth1_new_org_lag11_12_yyyy_week_nunique,auth1_new_org_lag11_12_purchase_date_max,auth1_new_org_lag11_12_purchase_date_min,auth1_new_org_lag11_12_month_diff_mean,auth1_new_org_lag11_12_month_diff_std,auth1_new_org_lag11_12_purchase_amount_sum,auth1_new_org_lag11_12_purchase_amount_max,auth1_new_org_lag11_12_purchase_amount_min,auth1_new_org_lag11_12_purchase_amount_mean,auth1_new_org_lag11_12_installments_sum,auth1_new_org_lag11_12_installments_max,auth1_new_org_lag11_12_installments_min,auth1_new_org_lag11_12_installments_mean,auth1_new_org_lag11_12_installments_std,auth1_new_org_lag11_12_merchant_id_nunique,auth1_new_org_lag11_12_merchant_category_id_nunique,auth1_new_org_lag11_12_card_id_size,auth1_new_org_lag11_12_city_id_nunique,auth1_new_org_lag11_12_state_id_nunique,auth1_new_org_lag11_12_subsector_id_nunique,auth1_new_org_lag11_12_this_term,auth1_new_org_lag11_12_freq_per_this_term,auth1_new_org_lag11_12_amount_per_this_term,auth1_new_org_lag11_12_instthisments_per_this_term,auth1_new_org_lag11_12_amount_per_installments_sum,auth1_new_org_lag11_12_amount_per_installments_mean,auth1_new_org_lag11_12_monthly_avg_category_2_1_0_mean,auth1_new_org_lag11_12_monthly_avg_category_2_2_0_mean,auth1_new_org_lag11_12_monthly_avg_category_2_3_0_mean,auth1_new_org_lag11_12_monthly_avg_category_2_4_0_mean,auth1_new_org_lag11_12_monthly_avg_category_2_5_0_mean,auth1_new_org_lag11_12_monthly_avg_category_3_A_mean,auth1_new_org_lag11_12_monthly_avg_category_3_B_mean,auth1_new_org_lag11_12_monthly_avg_category_3_C_mean,auth1_new_org_lag11_12_monthly_avg_category_1_sum,auth1_new_org_lag11_12_monthly_avg_category_1_mean,auth1_new_org_lag11_12_monthly_avg_month_lag_mean,auth1_new_org_lag11_12_monthly_avg_month_lag_std,auth1_new_org_lag11_12_monthly_avg_yyyy_week_nunique,auth1_new_org_lag11_12_monthly_avg_month_diff_mean,auth1_new_org_lag11_12_monthly_avg_month_diff_std,auth1_new_org_lag11_12_monthly_avg_purchase_amount_sum,auth1_new_org_lag11_12_monthly_avg_purchase_amount_max,auth1_new_org_lag11_12_monthly_avg_purchase_amount_min,auth1_new_org_lag11_12_monthly_avg_purchase_amount_mean,auth1_new_org_lag11_12_monthly_avg_installments_sum,auth1_new_org_lag11_12_monthly_avg_installments_max,auth1_new_org_lag11_12_monthly_avg_installments_min,auth1_new_org_lag11_12_monthly_avg_installments_mean,auth1_new_org_lag11_12_monthly_avg_installments_std,auth1_new_org_lag11_12_monthly_avg_merchant_id_nunique,auth1_new_org_lag11_12_monthly_avg_merchant_category_id_nunique,auth1_new_org_lag11_12_monthly_avg_card_id_size,auth1_new_org_lag11_12_monthly_avg_city_id_nunique,auth1_new_org_lag11_12_monthly_avg_state_id_nunique,auth1_new_org_lag11_12_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.000000,-28.00000,-15.000000,-1.0,NaT,NaT,584.000000,384.964033,-1.0,-1.000000,-1.000000,-1.000000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.000000,-28.0,-15.0,-1.0,584.000000,121.213203,-1.0,-1.000000,-1.000000,-1.00000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_3d0044924f,0.925926,0.0,0.0,0.0,0.0,0.0,0.740741,0.185185,2.0,0.074074,-0.62963,0.492103,8.0,2018-01-28 10:38:06,2017-12-06 14:24:30,118.962963,17.236994,2.0,0.314697,0.004517,0.074036,31.0,3.0,0.0,1.148438,0.601562,25.0,15.0,27.0,5.0,2.0,12.0,52.0,0.509434,0.037736,0.58490

In [28]:
#========================================================================
# Save Feature
#========================================================================

ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]

for df_feat, fname in zip(feat_list, ['l45', 'l56', 'l67', 'l78', 'l89', 'l910', 'l1011', 'l1112']):
    fname = '180_' + fname
    for col in df_feat.columns:
        if col in ignore_features: continue
        if (col.count('feature_')):continue
        if (col.count('purchase_date')) and not(col.count('diff')):continue
        feature = df_feat[col].fillna(-1).astype('float32').values
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
#         utils.to_pkl_gzip(path = f'../features/5_tmp/{fname}_{col}@', obj=feature)